# Big Data Assignment 4: White House DataFrame

Rules:

RULE-1: Header to be used as metadata

RULE-2: Convert column names into lowercase

RULE-3: If a given record is missing visitor 
        (just the last name of visitor) or 
        visitee (just the last name of visitee)
        , then that record is dropped from all calculations

RULE-4: All input must be converted to lowercase letters

RULE-5: Your solution has to be generic and 
        should be able to handle billions of records

RULE-6: You have to pass input as a parameter to your PySpark program

You may download the input from here: https://obamawhitehouse.archives.gov/sites/default/files/disclosures/whitehouse_waves-2016_12.csv_.zip

In [1]:
spark.version

'3.3.2'

In [2]:
spark

In [3]:
#used in terminal but not Jupyter Notebooks
# input_path = sys.argv[1]  
#    print("input_path: {}".format(input_path))

In [4]:
input_path='whitehouse_data.csv'

### Create pyspark DataFrame

In [5]:
df_start = spark\
          .read\
          .format("csv")\
          .option("header","true")\
          .option("inferSchema", "true")\
          .load(input_path) 

In [6]:
df_start.printSchema()

root
 |-- NAMELAST: string (nullable = true)
 |-- NAMEFIRST: string (nullable = true)
 |-- NAMEMID: string (nullable = true)
 |-- UIN: string (nullable = true)
 |-- BDGNBR: integer (nullable = true)
 |-- ACCESS_TYPE: string (nullable = true)
 |-- TOA: string (nullable = true)
 |-- POA: string (nullable = true)
 |-- TOD: string (nullable = true)
 |-- POD: string (nullable = true)
 |-- APPT_MADE_DATE: string (nullable = true)
 |-- APPT_START_DATE: string (nullable = true)
 |-- APPT_END_DATE: string (nullable = true)
 |-- APPT_CANCEL_DATE: string (nullable = true)
 |-- Total_People: integer (nullable = true)
 |-- LAST_UPDATEDBY: string (nullable = true)
 |-- POST: string (nullable = true)
 |-- LASTENTRYDATE: string (nullable = true)
 |-- TERMINAL_SUFFIX: string (nullable = true)
 |-- visitee_namelast: string (nullable = true)
 |-- visitee_namefirst: string (nullable = true)
 |-- MEETING_LOC: string (nullable = true)
 |-- MEETING_ROOM: string (nullable = true)
 |-- CALLER_NAME_LAST: string

In [7]:
df_start.createOrReplaceTempView("house_table")

23/05/25 12:49:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
query = "select NAMELAST, NAMEFIRST, NAMEMID, visitee_namelast, visitee_namefirst from house_table"

### Create dataframe with relevant columns

In [9]:
df_all = spark.sql(query)

In [10]:
df_all.createOrReplaceTempView("all_table")

In [11]:
not_null_query = "SELECT NAMELAST, NAMEFIRST, visitee_namelast, \
visitee_namefirst FROM all_table WHERE NAMELAST IS NOT NULL AND visitee_namelast IS NOT NULL"

### Create a dataframe with non null values and one with null values

In [12]:
df_not_null = spark.sql(not_null_query)

In [13]:
df_null = spark.sql("SELECT NAMELAST, NAMEFIRST, visitee_namelast, visitee_namefirst FROM all_table \
WHERE NAMELAST IS NULL OR visitee_namelast IS NULL")

In [14]:
df_all.count()

970504

In [15]:
df_not_null.count()

911249

In [16]:
df_null.count()

59255

Make sure nulls + non nulls add to total

In [17]:
df_not_null.count() + df_null.count()

970504

In [18]:
df_not_null.createOrReplaceTempView("pull_table")

In [19]:
low_col_q = "SELECT LOWER(NAMELAST) as namelast ,LOWER(NAMEFIRST) as namefirst,\
LOWER(visitee_namelast) as visitee_namelast,LOWER(visitee_namefirst) as visitee_namefirst FROM pull_table"

### Make all values and columns lowercase

In [20]:
lower_df = spark.sql(low_col_q)

In [21]:
lower_df.createOrReplaceTempView("low_table")

Create grouped by visitor dataframe to see most common visitors

In [22]:
df_visitor = spark.sql("SELECT namelast,namefirst, count(*) AS count \
FROM low_table GROUP BY namelast,namefirst ORDER BY count DESC")

In [23]:
df_visitor.createOrReplaceTempView("visitor_table")

Create grouped by visitee dataframe to see most common visitees

In [24]:
df_visitee = spark.sql("SELECT visitee_namelast,visitee_namefirst, count(*) AS count \
FROM low_table GROUP BY visitee_namelast,visitee_namefirst ORDER BY count DESC")

In [25]:
df_visitee.createOrReplaceTempView("visitee_table")

Create grouped by visitor and visitee dataframe to see most common meetings

In [26]:
vis_vis_df = spark.sql("SELECT namelast,namefirst,visitee_namelast,visitee_namefirst, count(*) AS count \
FROM low_table GROUP BY namelast,namefirst,visitee_namelast,visitee_namefirst ORDER BY count DESC")

In [27]:
vis_vis_df.createOrReplaceTempView("meet_table")

Take top 10 of both visitors, visitees, and meets

In [28]:
final_visitors_q = "SELECT * FROM visitor_table LIMIT 10"

In [29]:
final_visitees_q = "SELECT * FROM visitee_table LIMIT 10"

In [30]:
final_meets_q = "SELECT * FROM meet_table LIMIT 10"

In [31]:
top_visitors = spark.sql(final_visitors_q)

In [32]:
top_visitees = spark.sql(final_visitees_q)

In [33]:
top_meets = spark.sql(final_meets_q)

# Outputs:

In [34]:
top_visitors.show()

+--------+---------+-----+
|namelast|namefirst|count|
+--------+---------+-----+
| kidwell|   lauren|  222|
|  thomas| benjamin|  196|
|    haro|   steven|  183|
|  berner|katherine|  177|
|   grant|  patrick|  155|
|    haas|   jordan|  152|
|   garza|   steven|  127|
|   cohen|    mandy|  122|
|  martin|  kathryn|  122|
|   brown| jennifer|  117|
+--------+---------+-----+



In [35]:
top_visitees.show()

+----------------+-----------------+------+
|visitee_namelast|visitee_namefirst| count|
+----------------+-----------------+------+
|          office|         visitors|430881|
|           waves|   visitorsoffice| 44129|
|          bryant|             ruth| 13970|
|           oneil|           olivia| 13155|
|        thompson|            jared| 11618|
|               /|            potus| 10900|
|          burton|           collin|  9672|
|           megan|          matthew|  7944|
|        mayerson|            asher|  6886|
|      dessources|          kalisha|  5289|
+----------------+-----------------+------+



In [36]:
top_meets.show()

+--------+---------+----------------+-----------------+-----+
|namelast|namefirst|visitee_namelast|visitee_namefirst|count|
+--------+---------+----------------+-----------------+-----+
| kidwell|   lauren|        yudelson|             alex|  103|
|    haas|   jordan|        yudelson|             alex|   90|
|   grant|  patrick|        yudelson|             alex|   89|
|  thomas| benjamin|        yudelson|             alex|   89|
|   cohen|    mandy|         lambrew|           jeanne|   84|
|    haro|   steven|        yudelson|             alex|   84|
|  berner|katherine|        yudelson|             alex|   82|
|   roche|  shannon|        yudelson|             alex|   70|
|  urizar| jennifer|         johnson|            katie|   68|
|  martin|  kathryn|         lambrew|           jeanne|   61|
+--------+---------+----------------+-----------------+-----+



In [37]:
df_null.count()

59255